In [1]:
%%bash
conda install -y -c bioconda minimap2

Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - minimap2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.5.15  |                0         133 KB
    conda-4.6.14               |           py37_0         2.1 MB
    minimap2-2.17              |       h84994c4_0         2.0 MB  bioconda
    openssl-1.1.1c             |       h7b6447c_1         3.8 MB
    ------------------------------------------------------------
                                           Total:         8.1 MB

The following NEW packages will be INSTALLED:

  minimap2           bioconda/linux-64::minimap2-2.17-h84994c4_0

The following packages will be UPDATED:

  ca-certificates                               2019.1.23-0 --> 2019.5.15-0
  conda                    conda-forge::conda-4.6.13-py37_0 --> pkgs/main::conda-4.6.1

In [4]:
%%bash
## data source: ftp://ftp.ebi.ac.uk:/pub/databases/ipd/imgt/hla/fasta/hla_gen.fasta
wget ftp://ftp.ebi.ac.uk:/pub/databases/ipd/imgt/hla/fasta/hla_gen.fasta
less hla_gen.fasta | tr " " "_" > tmp
mv tmp hla_gen.fasta

--2019-06-10 14:57:39--  ftp://ftp.ebi.ac.uk/pub/databases/ipd/imgt/hla/fasta/hla_gen.fasta
           => ‘hla_gen.fasta’
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.192.4
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.192.4|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/databases/ipd/imgt/hla/fasta ... done.
==> SIZE hla_gen.fasta ... 31008738
==> PASV ... done.    ==> RETR hla_gen.fasta ... done.
Length: 31008738 (30M) (unauthoritative)

     0K .......... .......... .......... .......... ..........  0%  267K 1m53s
    50K .......... .......... .......... .......... ..........  0%  647K 80s
   100K .......... .......... .......... .......... ..........  0%  112M 53s
   150K .......... .......... .......... .......... ..........  0%  357M 40s
   200K .......... .......... .......... .......... ..........  0%  626K 41s
   250K .......... .......... .......... .......... ..........  0%  16

In [5]:
%%bash
dx download -f --recursive /Haplotype_ASM_0525_2019/
ls Haplotype_ASM_0525_2019

p_ctg_cns_H1.fa
p_ctg_cns_H2.fa


In [6]:
%%bash
minimap2 -x asm5 -c --eqx  Haplotype_ASM_0525_2019/p_ctg_cns_H1.fa  hla_gen.fasta > hla_gen_H1.paf &
minimap2 -x asm5 -c --eqx  Haplotype_ASM_0525_2019/p_ctg_cns_H2.fa  hla_gen.fasta > hla_gen_H2.paf &


[M::mm_idx_gen::0.126*0.99] collected minimizers
[M::mm_idx_gen::0.128*1.00] collected minimizers
[M::mm_idx_gen::0.144*1.20] sorted minimizers
[M::main::0.144*1.20] loaded/built the index for 7 target sequence(s)
[M::mm_idx_gen::0.148*1.19] sorted minimizers
[M::main::0.148*1.19] loaded/built the index for 10 target sequence(s)
[M::mm_mapopt_update::0.153*1.17] mid_occ = 100
[M::mm_idx_stat] kmer size: 19; skip: 19; is_hpc: 0; #seq: 7
[M::mm_mapopt_update::0.157*1.17] mid_occ = 100
[M::mm_idx_stat] kmer size: 19; skip: 19; is_hpc: 0; #seq: 10
[M::mm_idx_stat::0.160*1.18] distinct minimizers: 454308 (94.97% are singletons); average occurrences: 1.149; average spacing: 9.950
[M::mm_idx_stat::0.163*1.18] distinct minimizers: 456455 (94.74% are singletons); average occurrences: 1.151; average spacing: 9.953
[M::worker_pipeline::4.173*2.87] mapped 7260 sequences
[M::main] Version: 2.17-r941
[M::main] CMD: minimap2 -x asm5 -c --eqx Haplotype_ASM_0525_2019/p_ctg_cns_H1.fa hla_gen.fasta
[M::m

In [47]:
def get_allele_to_hits(fn):
    allele_to_hits = {}
    with open(fn)  as f:
        for row in f:
            if row[0] in ["#","@"]: 
                continue
            row = row.strip().split("\t")
            HLA_allele = row[0].split("_")[1].split("*")[0]

            data = [c.split(":") for c in row[13:]]
            data = dict( ((c[0],c[2]) for c in data) )
   
            allele_to_hits.setdefault(HLA_allele, [])
            allele_to_hits[HLA_allele].append( (float(data["de"]), row[0], data["cg"], row[5], int(row[7]), row[4] ) ) 
    return allele_to_hits

In [56]:
with open("HLA-ALN-H1.tsv","w") as f:
    for allele, hits in get_allele_to_hits("hla_gen_H1.paf").items():
        hits.sort(key=lambda x: x[0])
        for h in hits[:5]:
            print(allele, *[str(c) for c in h], sep="\t", file=f)

In [57]:
with open("HLA-ALN-H2.tsv","w") as f:
    for allele, hits in get_allele_to_hits("hla_gen_H2.paf").items():
        hits.sort(key=lambda x: x[0])
        for h in hits[:5]:
            print(allele, *[str(c) for c in h], sep="\t", file=f)
        print()